# Vectorization in Deep Neural Networks

## 🎯 The Power of Vectorization

**Question**: Why do we use matrix operations instead of loops in neural networks?

**Answer**: Vectorization makes neural networks **10-100x faster** and enables efficient batch processing!

---

## 📋 What This Notebook Covers

1. ✅ What is vectorization and why it matters
2. ✅ Vectorization basics with NumPy
3. ✅ **Mathematical formulation of forward propagation (vectorized)**
4. ✅ **Mathematical formulation of backpropagation (vectorized)**
5. ✅ Complete implementation: loops vs vectorized
6. ✅ Performance benchmarks
7. ✅ Batch processing and mini-batch gradient descent
8. ✅ Dimension analysis and debugging tips

Let's unlock the power of vectorization! 🚀

## 1. Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from IPython.display import display, Markdown

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")

## 2. What is Vectorization?

**Vectorization** is the process of replacing explicit loops with array operations.

### Example: Computing dot product

**Non-vectorized (Loop)**:
```python
result = 0
for i in range(len(a)):
    result += a[i] * b[i]
```

**Vectorized (NumPy)**:
```python
result = np.dot(a, b)
```

The vectorized version is:
- ✅ **Faster** (uses optimized C/Fortran libraries)
- ✅ **Cleaner** (more readable code)
- ✅ **Parallelizable** (can use SIMD, GPU acceleration)

### Demo: Speed Comparison

In [ ]:
# Create large arrays
n = 1000000
a = np.random.randn(n)
b = np.random.randn(n)

# Method 1: Loop (non-vectorized)
start = time.time()
result_loop = 0
for i in range(n):
    result_loop += a[i] * b[i]
time_loop = time.time() - start

# Method 2: Vectorized
start = time.time()
result_vectorized = np.dot(a, b)
time_vectorized = time.time() - start

print("⏱️  Performance Comparison:")
print(f"\nLoop version:       {time_loop*1000:.2f} ms")
print(f"Vectorized version: {time_vectorized*1000:.2f} ms")
print(f"\n🚀 Speedup: {time_loop/time_vectorized:.1f}x faster!")
print(f"\nResults match: {np.allclose(result_loop, result_vectorized)}")

## 3. Vectorization Basics

### Key NumPy Operations

In [ ]:
# Element-wise operations
a = np.array([1, 2, 3, 4])
b = np.array([5, 6, 7, 8])

print("Element-wise Operations:")
print(f"a + b = {a + b}")
print(f"a * b = {a * b}")
print(f"a ** 2 = {a ** 2}")

# Matrix multiplication
A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])

print("\nMatrix Multiplication:")
print(f"A @ B =\n{A @ B}")

# Broadcasting
C = np.array([[1, 2, 3], [4, 5, 6]])
d = np.array([10, 20, 30])

print("\nBroadcasting:")
print(f"C =\n{C}")
print(f"d = {d}")
print(f"C + d =\n{C + d}")

### Broadcasting Rules

NumPy automatically broadcasts arrays of different shapes:

```
Shape (2, 3) + Shape (3,)   → Shape (2, 3)  ✅
Shape (2, 3) + Shape (2, 1) → Shape (2, 3)  ✅
Shape (2, 3) + Shape (2,)   → Error         ❌
```

**Rule**: Dimensions are compatible if:
1. They are equal, OR
2. One of them is 1

In [ ]:
# Broadcasting examples
print("Broadcasting Examples:\n")

# Example 1: Add scalar to matrix
A = np.array([[1, 2, 3], [4, 5, 6]])
print(f"A + 10 =\n{A + 10}\n")

# Example 2: Add row vector to matrix
row = np.array([10, 20, 30])
print(f"A + row =\n{A + row}\n")

# Example 3: Add column vector to matrix
col = np.array([[100], [200]])
print(f"A + col =\n{A + col}")

## 4. Neural Network Notation

Before diving into vectorization, let's establish notation:

### Single Sample Notation

For a single training example:

- $x \in \mathbb{R}^{n_x}$ - Input features (column vector)
- $W^{[l]} \in \mathbb{R}^{n^{[l]} \times n^{[l-1]}}$ - Weight matrix for layer $l$
- $b^{[l]} \in \mathbb{R}^{n^{[l]}}$ - Bias vector for layer $l$
- $z^{[l]} \in \mathbb{R}^{n^{[l]}}$ - Linear activation for layer $l$
- $a^{[l]} \in \mathbb{R}^{n^{[l]}}$ - Activation output for layer $l$

### Batch (Vectorized) Notation

For $m$ training examples:

- $X \in \mathbb{R}^{n_x \times m}$ - Input matrix (each column is one example)
- $Z^{[l]} \in \mathbb{R}^{n^{[l]} \times m}$ - Linear activations for all examples
- $A^{[l]} \in \mathbb{R}^{n^{[l]} \times m}$ - Activation outputs for all examples

**Key Insight**: Capital letters denote matrices containing multiple examples!

## 5. Forward Propagation: Mathematical Formulation

### Single Sample (Non-Vectorized)

For one training example $x^{(i)}$:

$$
\begin{align}
z^{[l](i)} &= W^{[l]} a^{[l-1](i)} + b^{[l]} \quad \text{(Linear transformation)} \\
a^{[l](i)} &= g^{[l]}(z^{[l](i)}) \quad \text{(Activation function)}
\end{align}
$$

Where:
- $a^{[0](i)} = x^{(i)}$ (input)
- $g^{[l]}$ is the activation function (ReLU, tanh, sigmoid, etc.)

### Vectorized Batch Form

For $m$ training examples simultaneously:

$$
\begin{align}
Z^{[l]} &= W^{[l]} A^{[l-1]} + b^{[l]} \quad \text{Shape: } (n^{[l]}, m) \\
A^{[l]} &= g^{[l]}(Z^{[l]}) \quad \text{Shape: } (n^{[l]}, m)
\end{align}
$$

Where:
- $A^{[0]} = X$ (input matrix)
- $b^{[l]}$ is broadcast across all $m$ examples
- Each column of $Z^{[l]}$ and $A^{[l]}$ corresponds to one training example

### Dimension Analysis

$$
\underbrace{Z^{[l]}}_{(n^{[l]}, m)} = \underbrace{W^{[l]}}_{(n^{[l]}, n^{[l-1]})} \underbrace{A^{[l-1]}}_{(n^{[l-1]}, m)} + \underbrace{b^{[l]}}_{(n^{[l]}, 1)}
$$

The bias $b^{[l]}$ is broadcast to shape $(n^{[l]}, m)$ automatically!

## 6. Backpropagation: Mathematical Formulation

### Single Sample Gradients

For one training example, the gradients are:

**Output Layer** ($l = L$):
$$
dz^{[L](i)} = a^{[L](i)} - y^{(i)} \quad \text{(for cross-entropy loss)}
$$

**Hidden Layers** ($l = L-1, L-2, \ldots, 1$):
$$
\begin{align}
dz^{[l](i)} &= (W^{[l+1]})^T dz^{[l+1](i)} \odot g'^{[l]}(z^{[l](i)}) \\
dW^{[l]} &= dz^{[l](i)} (a^{[l-1](i)})^T \\
db^{[l]} &= dz^{[l](i)} \\
da^{[l-1](i)} &= (W^{[l]})^T dz^{[l](i)}
\end{align}
$$

Where $\odot$ denotes element-wise multiplication.

### Vectorized Batch Gradients

For $m$ training examples simultaneously:

**Output Layer** ($l = L$):
$$
dZ^{[L]} = A^{[L]} - Y \quad \text{Shape: } (n^{[L]}, m)
$$

**Hidden Layers** ($l = L-1, L-2, \ldots, 1$):
$$
\begin{align}
dZ^{[l]} &= (W^{[l+1]})^T dZ^{[l+1]} \odot g'^{[l]}(Z^{[l]}) \quad \text{Shape: } (n^{[l]}, m) \\
dW^{[l]} &= \frac{1}{m} dZ^{[l]} (A^{[l-1]})^T \quad \text{Shape: } (n^{[l]}, n^{[l-1]}) \\
db^{[l]} &= \frac{1}{m} \sum_{i=1}^{m} dZ^{[l]} \quad \text{Shape: } (n^{[l]}, 1) \\
dA^{[l-1]} &= (W^{[l]})^T dZ^{[l]} \quad \text{Shape: } (n^{[l-1]}, m)
\end{align}
$$

### Key Observations

1. **Averaging**: We divide by $m$ when computing $dW^{[l]}$ and $db^{[l]}$ because we're averaging the gradients across all training examples.

2. **Bias Gradient**: $db^{[l]} = \frac{1}{m} \sum_{i=1}^{m} dZ^{[l]}$ sums across the columns (axis=1) and averages.

3. **Matrix Dimensions**: The transpose operations ensure dimensions match correctly.

4. **Efficiency**: Computing gradients for all $m$ examples at once is much faster than looping!

## 7. Dimension Analysis Table

Understanding dimensions is crucial for debugging. Here's a complete reference:

| Variable | Single Sample | Batch (m samples) | Description |
|----------|---------------|-------------------|-------------|
| $x$ | $(n_x, 1)$ | $X: (n_x, m)$ | Input features |
| $W^{[l]}$ | $(n^{[l]}, n^{[l-1]})$ | $(n^{[l]}, n^{[l-1]})$ | Weights (same for batch) |
| $b^{[l]}$ | $(n^{[l]}, 1)$ | $(n^{[l]}, 1)$ | Bias (broadcast to m) |
| $z^{[l]}$ | $(n^{[l]}, 1)$ | $Z^{[l]}: (n^{[l]}, m)$ | Linear activation |
| $a^{[l]}$ | $(n^{[l]}, 1)$ | $A^{[l]}: (n^{[l]}, m)$ | Activation output |
| $dz^{[l]}$ | $(n^{[l]}, 1)$ | $dZ^{[l]}: (n^{[l]}, m)$ | Gradient of z |
| $dW^{[l]}$ | $(n^{[l]}, n^{[l-1]})$ | $(n^{[l]}, n^{[l-1]})$ | Gradient of W |
| $db^{[l]}$ | $(n^{[l]}, 1)$ | $(n^{[l]}, 1)$ | Gradient of b |

**Memory Tip**: 
- Weights and biases have the same shape for single/batch
- Activations and gradients gain an extra dimension $m$ for batches
- Each column represents one training example

## 8. Implementation: Loop vs Vectorized

Let's implement a simple 2-layer neural network both ways to see the difference!

In [ ]:
# Generate synthetic data
np.random.seed(42)

# Dataset: 100 samples, 3 features, binary classification
m = 100  # number of samples
n_x = 3  # input features
n_h = 4  # hidden layer size
n_y = 1  # output size

# Generate random data
X = np.random.randn(n_x, m)
Y = (np.random.randn(n_y, m) > 0).astype(float)

# Initialize parameters
W1 = np.random.randn(n_h, n_x) * 0.01
b1 = np.zeros((n_h, 1))
W2 = np.random.randn(n_y, n_h) * 0.01
b2 = np.zeros((n_y, 1))

print(f"Dataset: {m} samples, {n_x} features")
print(f"Architecture: {n_x} → {n_h} → {n_y}")
print(f"\nShapes:")
print(f"  X: {X.shape}")
print(f"  Y: {Y.shape}")
print(f"  W1: {W1.shape}, b1: {b1.shape}")
print(f"  W2: {W2.shape}, b2: {b2.shape}")

### Activation Functions

In [ ]:
def sigmoid(z):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

def sigmoid_derivative(z):
    """Derivative of sigmoid."""
    s = sigmoid(z)
    return s * (1 - s)

def relu(z):
    """ReLU activation function."""
    return np.maximum(0, z)

def relu_derivative(z):
    """Derivative of ReLU."""
    return (z > 0).astype(float)

print("✅ Activation functions defined!")

### Method 1: Loop-Based (Non-Vectorized)

Process one example at a time using loops.

In [ ]:
def forward_propagation_loop(X, Y, W1, b1, W2, b2):
    """
    Forward propagation using loops (one example at a time).
    
    This is SLOW but easier to understand.
    """
    m = X.shape[1]
    
    # Initialize outputs
    Z1 = np.zeros((W1.shape[0], m))
    A1 = np.zeros((W1.shape[0], m))
    Z2 = np.zeros((W2.shape[0], m))
    A2 = np.zeros((W2.shape[0], m))
    
    # Loop over each training example
    for i in range(m):
        # Get single example
        x = X[:, i:i+1]  # Shape: (n_x, 1)
        
        # Layer 1
        z1 = W1 @ x + b1
        a1 = relu(z1)
        
        # Layer 2
        z2 = W2 @ a1 + b2
        a2 = sigmoid(z2)
        
        # Store results
        Z1[:, i:i+1] = z1
        A1[:, i:i+1] = a1
        Z2[:, i:i+1] = z2
        A2[:, i:i+1] = a2
    
    # Compute loss
    loss = -np.mean(Y * np.log(A2 + 1e-8) + (1 - Y) * np.log(1 - A2 + 1e-8))
    
    cache = {'Z1': Z1, 'A1': A1, 'Z2': Z2, 'A2': A2}
    return A2, loss, cache

def backward_propagation_loop(X, Y, W1, b1, W2, b2, cache):
    """
    Backward propagation using loops (one example at a time).
    
    This is SLOW but easier to understand.
    """
    m = X.shape[1]
    Z1, A1, Z2, A2 = cache['Z1'], cache['A1'], cache['Z2'], cache['A2']
    
    # Initialize gradients
    dW1 = np.zeros_like(W1)
    db1 = np.zeros_like(b1)
    dW2 = np.zeros_like(W2)
    db2 = np.zeros_like(b2)
    
    # Loop over each training example
    for i in range(m):
        # Get single example
        x = X[:, i:i+1]
        y = Y[:, i:i+1]
        a1 = A1[:, i:i+1]
        a2 = A2[:, i:i+1]
        z1 = Z1[:, i:i+1]
        
        # Backprop through layer 2
        dz2 = a2 - y
        dW2 += dz2 @ a1.T
        db2 += dz2
        
        # Backprop through layer 1
        da1 = W2.T @ dz2
        dz1 = da1 * relu_derivative(z1)
        dW1 += dz1 @ x.T
        db1 += dz1
    
    # Average gradients
    dW1 /= m
    db1 /= m
    dW2 /= m
    db2 /= m
    
    return dW1, db1, dW2, db2

print("✅ Loop-based functions defined!")

### Method 2: Vectorized

Process all examples simultaneously using matrix operations.

In [ ]:
def forward_propagation_vectorized(X, Y, W1, b1, W2, b2):
    """
    Forward propagation using vectorization (all examples at once).
    
    This is FAST and efficient!
    """
    # Layer 1: Z1 = W1 @ X + b1
    Z1 = W1 @ X + b1  # Shape: (n_h, m)
    A1 = relu(Z1)      # Shape: (n_h, m)
    
    # Layer 2: Z2 = W2 @ A1 + b2
    Z2 = W2 @ A1 + b2  # Shape: (n_y, m)
    A2 = sigmoid(Z2)   # Shape: (n_y, m)
    
    # Compute loss
    loss = -np.mean(Y * np.log(A2 + 1e-8) + (1 - Y) * np.log(1 - A2 + 1e-8))
    
    cache = {'Z1': Z1, 'A1': A1, 'Z2': Z2, 'A2': A2}
    return A2, loss, cache

def backward_propagation_vectorized(X, Y, W1, b1, W2, b2, cache):
    """
    Backward propagation using vectorization (all examples at once).
    
    This is FAST and efficient!
    """
    m = X.shape[1]
    Z1, A1, Z2, A2 = cache['Z1'], cache['A1'], cache['Z2'], cache['A2']
    
    # Backprop through layer 2
    dZ2 = A2 - Y                    # Shape: (n_y, m)
    dW2 = (1/m) * (dZ2 @ A1.T)      # Shape: (n_y, n_h)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)  # Shape: (n_y, 1)
    
    # Backprop through layer 1
    dA1 = W2.T @ dZ2                # Shape: (n_h, m)
    dZ1 = dA1 * relu_derivative(Z1) # Shape: (n_h, m)
    dW1 = (1/m) * (dZ1 @ X.T)       # Shape: (n_h, n_x)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)  # Shape: (n_h, 1)
    
    return dW1, db1, dW2, db2

print("✅ Vectorized functions defined!")

### Performance Comparison

In [ ]:
# Test loop-based version
start = time.time()
A2_loop, loss_loop, cache_loop = forward_propagation_loop(X, Y, W1, b1, W2, b2)
dW1_loop, db1_loop, dW2_loop, db2_loop = backward_propagation_loop(X, Y, W1, b1, W2, b2, cache_loop)
time_loop = time.time() - start

# Test vectorized version
start = time.time()
A2_vec, loss_vec, cache_vec = forward_propagation_vectorized(X, Y, W1, b1, W2, b2)
dW1_vec, db1_vec, dW2_vec, db2_vec = backward_propagation_vectorized(X, Y, W1, b1, W2, b2, cache_vec)
time_vec = time.time() - start

print("⏱️  Performance Comparison:\n")
print(f"Loop-based:  {time_loop*1000:.2f} ms")
print(f"Vectorized:  {time_vec*1000:.2f} ms")
print(f"\n🚀 Speedup: {time_loop/time_vec:.1f}x faster!")

print("\n✅ Verification (results should match):")
print(f"Loss match: {np.allclose(loss_loop, loss_vec)}")
print(f"dW1 match:  {np.allclose(dW1_loop, dW1_vec)}")
print(f"dW2 match:  {np.allclose(dW2_loop, dW2_vec)}")
print(f"db1 match:  {np.allclose(db1_loop, db1_vec)}")
print(f"db2 match:  {np.allclose(db2_loop, db2_vec)}")

## 9. Scaling Test: Different Batch Sizes

Let's see how performance scales with batch size!

In [ ]:
batch_sizes = [10, 50, 100, 200, 500, 1000]
times_loop = []
times_vec = []

for m in batch_sizes:
    # Generate data
    X_test = np.random.randn(n_x, m)
    Y_test = (np.random.randn(n_y, m) > 0).astype(float)
    
    # Time loop version
    start = time.time()
    A2, loss, cache = forward_propagation_loop(X_test, Y_test, W1, b1, W2, b2)
    dW1, db1, dW2, db2 = backward_propagation_loop(X_test, Y_test, W1, b1, W2, b2, cache)
    times_loop.append(time.time() - start)
    
    # Time vectorized version
    start = time.time()
    A2, loss, cache = forward_propagation_vectorized(X_test, Y_test, W1, b1, W2, b2)
    dW1, db1, dW2, db2 = backward_propagation_vectorized(X_test, Y_test, W1, b1, W2, b2, cache)
    times_vec.append(time.time() - start)

# Plot results
plt.figure(figsize=(12, 6))

plt.plot(batch_sizes, np.array(times_loop)*1000, 'o-', linewidth=2, 
         markersize=8, label='Loop-based', color='#FF6B6B')
plt.plot(batch_sizes, np.array(times_vec)*1000, 's-', linewidth=2, 
         markersize=8, label='Vectorized', color='#4ECDC4')

plt.xlabel('Batch Size (m)', fontsize=12, fontweight='bold')
plt.ylabel('Time (ms)', fontsize=12, fontweight='bold')
plt.title('Performance Scaling: Loop vs Vectorized', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Speedup by Batch Size:")
for i, m in enumerate(batch_sizes):
    speedup = times_loop[i] / times_vec[i]
    print(f"  m={m:4d}: {speedup:.1f}x faster")

## 10. Complete Training Example

Let's train a neural network using vectorized operations!

In [ ]:
# Generate a more interesting dataset
np.random.seed(42)
m = 300
n_x = 2
n_h = 10
n_y = 1

# Create spiral dataset
theta = np.linspace(0, 4*np.pi, m//2)
r = np.linspace(0.5, 2, m//2)

# Class 0: spiral
X_class0 = np.vstack([r * np.cos(theta), r * np.sin(theta)])
Y_class0 = np.zeros((1, m//2))

# Class 1: spiral (rotated)
X_class1 = np.vstack([r * np.cos(theta + np.pi), r * np.sin(theta + np.pi)])
Y_class1 = np.ones((1, m//2))

# Combine
X = np.hstack([X_class0, X_class1])
Y = np.hstack([Y_class0, Y_class1])

# Add noise
X += np.random.randn(*X.shape) * 0.1

# Shuffle
indices = np.random.permutation(m)
X = X[:, indices]
Y = Y[:, indices]

print(f"Dataset: {m} samples, {n_x} features")
print(f"Architecture: {n_x} → {n_h} → {n_y}")

In [ ]:
# Visualize dataset
plt.figure(figsize=(10, 8))
plt.scatter(X[0, Y[0]==0], X[1, Y[0]==0], c='#FF6B6B', s=50, 
           alpha=0.7, edgecolors='black', linewidth=1, label='Class 0')
plt.scatter(X[0, Y[0]==1], X[1, Y[0]==1], c='#4ECDC4', s=50, 
           alpha=0.7, edgecolors='black', linewidth=1, label='Class 1')
plt.xlabel('Feature 1', fontsize=12, fontweight='bold')
plt.ylabel('Feature 2', fontsize=12, fontweight='bold')
plt.title('Spiral Dataset', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
# Initialize parameters
np.random.seed(42)
W1 = np.random.randn(n_h, n_x) * 0.1
b1 = np.zeros((n_h, 1))
W2 = np.random.randn(n_y, n_h) * 0.1
b2 = np.zeros((n_y, 1))

# Training parameters
learning_rate = 0.5
epochs = 2000
losses = []

print("🚀 Training Neural Network with Vectorization...\n")

# Training loop
for epoch in range(epochs):
    # Forward propagation (vectorized!)
    A2, loss, cache = forward_propagation_vectorized(X, Y, W1, b1, W2, b2)
    losses.append(loss)
    
    # Backward propagation (vectorized!)
    dW1, db1, dW2, db2 = backward_propagation_vectorized(X, Y, W1, b1, W2, b2, cache)
    
    # Update parameters
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    
    # Print progress
    if (epoch + 1) % 200 == 0:
        accuracy = np.mean((A2 > 0.5) == Y)
        print(f"Epoch {epoch+1:4d}: Loss = {loss:.4f}, Accuracy = {accuracy:.4f}")

print("\n✅ Training Complete!")

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(losses, linewidth=2, color='#FF6B6B')
plt.xlabel('Epoch', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=12, fontweight='bold')
plt.title('Training Loss Over Time', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize decision boundary
def plot_decision_boundary(X, Y, W1, b1, W2, b2):
    # Create mesh
    x_min, x_max = X[0, :].min() - 0.5, X[0, :].max() + 0.5
    y_min, y_max = X[1, :].min() - 0.5, X[1, :].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    # Predict on mesh
    X_mesh = np.c_[xx.ravel(), yy.ravel()].T
    A2, _, _ = forward_propagation_vectorized(X_mesh, None, W1, b1, W2, b2)
    Z = A2.reshape(xx.shape)
    
    # Plot
    plt.figure(figsize=(12, 9))
    plt.contourf(xx, yy, Z, levels=20, cmap='RdYlBu_r', alpha=0.6)
    plt.colorbar(label='P(y=1)')
    plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=3, linestyles='--')
    
    plt.scatter(X[0, Y[0]==0], X[1, Y[0]==0], c='#FF6B6B', s=80, 
               alpha=0.8, edgecolors='black', linewidth=1.5, label='Class 0')
    plt.scatter(X[0, Y[0]==1], X[1, Y[0]==1], c='#4ECDC4', s=80, 
               alpha=0.8, edgecolors='black', linewidth=1.5, label='Class 1')
    
    plt.xlabel('Feature 1', fontsize=12, fontweight='bold')
    plt.ylabel('Feature 2', fontsize=12, fontweight='bold')
    plt.title('Decision Boundary (Trained with Vectorization)', fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

plot_decision_boundary(X, Y, W1, b1, W2, b2)

# Final accuracy
A2_final, _, _ = forward_propagation_vectorized(X, Y, W1, b1, W2, b2)
accuracy = np.mean((A2_final > 0.5) == Y)
print(f"\n✅ Final Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

## 11. Mini-Batch Gradient Descent

In practice, we often use **mini-batches** instead of the full dataset:

- **Batch Gradient Descent**: Use all $m$ examples (what we did above)
- **Stochastic Gradient Descent (SGD)**: Use 1 example at a time
- **Mini-Batch Gradient Descent**: Use batches of size $b$ (e.g., 32, 64, 128)

### Why Mini-Batches?

1. **Memory Efficiency**: Can't fit millions of examples in memory
2. **Faster Convergence**: More frequent updates
3. **Better Generalization**: Noise in gradients acts as regularization
4. **GPU Optimization**: Modern GPUs are optimized for batch sizes like 32, 64, 128

In [ ]:
def create_mini_batches(X, Y, batch_size):
    """
    Create mini-batches from dataset.
    
    Parameters:
    - X: Input data (n_x, m)
    - Y: Labels (n_y, m)
    - batch_size: Size of each mini-batch
    
    Returns:
    - List of (X_batch, Y_batch) tuples
    """
    m = X.shape[1]
    mini_batches = []
    
    # Shuffle data
    permutation = np.random.permutation(m)
    X_shuffled = X[:, permutation]
    Y_shuffled = Y[:, permutation]
    
    # Create mini-batches
    num_complete_batches = m // batch_size
    
    for k in range(num_complete_batches):
        X_batch = X_shuffled[:, k*batch_size:(k+1)*batch_size]
        Y_batch = Y_shuffled[:, k*batch_size:(k+1)*batch_size]
        mini_batches.append((X_batch, Y_batch))
    
    # Handle remaining examples
    if m % batch_size != 0:
        X_batch = X_shuffled[:, num_complete_batches*batch_size:]
        Y_batch = Y_shuffled[:, num_complete_batches*batch_size:]
        mini_batches.append((X_batch, Y_batch))
    
    return mini_batches

# Example: Create mini-batches
batch_size = 32
mini_batches = create_mini_batches(X, Y, batch_size)

print(f"Dataset size: {X.shape[1]} examples")
print(f"Batch size: {batch_size}")
print(f"Number of mini-batches: {len(mini_batches)}")
print(f"\nMini-batch shapes:")
for i, (X_batch, Y_batch) in enumerate(mini_batches[:3]):
    print(f"  Batch {i+1}: X={X_batch.shape}, Y={Y_batch.shape}")

## 12. Summary: Key Takeaways

### What We Learned

✅ **Vectorization Basics**
- Replace loops with matrix operations
- 10-100x faster than loops
- Enables GPU acceleration

✅ **Forward Propagation (Vectorized)**
$$
\begin{align}
Z^{[l]} &= W^{[l]} A^{[l-1]} + b^{[l]} \quad \text{Shape: } (n^{[l]}, m) \\
A^{[l]} &= g^{[l]}(Z^{[l]}) \quad \text{Shape: } (n^{[l]}, m)
\end{align}
$$

✅ **Backpropagation (Vectorized)**
$$
\begin{align}
dZ^{[l]} &= (W^{[l+1]})^T dZ^{[l+1]} \odot g'^{[l]}(Z^{[l]}) \\
dW^{[l]} &= \frac{1}{m} dZ^{[l]} (A^{[l-1]})^T \\
db^{[l]} &= \frac{1}{m} \sum_{i=1}^{m} dZ^{[l]}
\end{align}
$$

✅ **Key Insights**
- Each column represents one training example
- Bias is broadcast across all examples
- Gradients are averaged over the batch ($\frac{1}{m}$)
- Matrix dimensions must match correctly

✅ **Practical Benefits**
- Faster training (10-100x speedup)
- Cleaner, more readable code
- Enables mini-batch gradient descent
- GPU acceleration

### Dimension Cheat Sheet

For a network with $L$ layers and batch size $m$:

| Variable | Shape | Description |
|----------|-------|-------------|
| $X$ | $(n_x, m)$ | Input features |
| $W^{[l]}$ | $(n^{[l]}, n^{[l-1]})$ | Weights layer $l$ |
| $b^{[l]}$ | $(n^{[l]}, 1)$ | Bias layer $l$ |
| $Z^{[l]}$ | $(n^{[l]}, m)$ | Linear activation |
| $A^{[l]}$ | $(n^{[l]}, m)$ | Activation output |
| $dZ^{[l]}$ | $(n^{[l]}, m)$ | Gradient of $Z^{[l]}$ |
| $dW^{[l]}$ | $(n^{[l]}, n^{[l-1]})$ | Gradient of $W^{[l]}$ |
| $db^{[l]}$ | $(n^{[l]}, 1)$ | Gradient of $b^{[l]}$ |

### Connection to Other Notebooks

This notebook extends concepts from:
- **`4_1_a_i_imp_three_layer_math_summary_of_gradient_derivation_explained.md`**: Single sample gradients → Batch gradients
- **`4_1_a_iv_n_layer_general_activation_gradient_derivation.md`**: General backpropagation formulas
- **`5_a_logistic_regression_pure_implementation.ipynb`**: Simple example of vectorization

### Next Steps

🚀 **Practice**:
- Implement your own vectorized neural network
- Experiment with different batch sizes
- Try different activation functions
- Add more layers

---

**🎓 Congratulations!** You now understand vectorization in deep neural networks, including the complete mathematical formulation of forward and backward propagation!